In [ ]:
#################################### This script extract and format meteo data ###############################

In [1]:
# Load GEE
import ee # Import the Python API package
import geemap # The geemap Python package is built upon the ipyleaflet and folium packages and implements several methods for interacting with Earth Engine data layer
ee.Authenticate() # Authenticate to the Earth Engine servers
ee.Initialize() # Initialize the API

Enter verification code:  4/1AanRRrvK9FkORVvfIPwWfoJrBcyZG-kiu5tSNqhIYRUx0KKG38NAiSEwCaU



Successfully saved authorization token.


In [112]:
# Point to sample corresponding to the Bernadouze peatland
PtsToSample = ee.Geometry.Point([1.4238939982481602,42.80272858095022])

# Create the map
Map1 = geemap.Map(center=[42.80272858095022,1.4238939982481602], basemap='HYBRID', zoom=18)
Map1.addLayer(PtsToSample, {},'Sampling points')
Map1

Map(center=[42.80272858095022, 1.4238939982481602], controls=(WidgetControl(options=['position', 'transparent_…

In [3]:
# Loading meteo data
PAR = ee.ImageCollection('MODIS/061/MCD18C2').filter(ee.Filter.date('2023-01-01', '2024-01-01')).select('GMT_1200_PAR').toBands()
TEMP = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('2023-01-01', '2024-01-01')).select('temperature_2m').toBands()

In [4]:
# Sampling and exporting to drive
TempSampled = TEMP.sampleRegions(collection= PtsToSample,scale= 1000,geometries=False)
taskParams = {'driveFolder': 'BERNADOUZE','fileFormat': 'CSV'}
task = ee.batch.Export.table.toDrive(TempSampled,str('TempSampled'),**taskParams)
task.start()

PARSampled = PAR.sampleRegions(collection= PtsToSample,scale= 1000,geometries=False)
taskParams = {'driveFolder': 'BERNADOUZE','fileFormat': 'CSV'}
task = ee.batch.Export.table.toDrive(PARSampled,str('PARSampled'),**taskParams)
task.start()

In [110]:
# Open resulting data from local directory
# Import modules
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

# formating TEMP
TEMPdata = pd.read_csv('F:/Redaction/Bernadouze/Meteo/TempSampled.csv',sep=',', index_col=0).T
TEMPdata.drop(TEMPdata.tail(1).index,inplace=True) # drop last n rows
TEMPdata = TEMPdata.rename(columns={"0_0": "TEMP_K"})
TEMPdata = TEMPdata.reset_index()
TEMPdata = TEMPdata.rename(columns={"index": "text"})
TEMPdata['year'] = TEMPdata['text'].str[:4]
TEMPdata['month'] = TEMPdata['text'].str[4:6]
TEMPdata['day'] = TEMPdata['text'].str[6:8]
TEMPdata = TEMPdata.drop(['text'], axis=1)

# formating PAR
PARdata = pd.read_csv('F:/Redaction/Bernadouze/Meteo/PARSampled.csv',sep=',', index_col=0).T
PARdata.drop(PARdata.tail(1).index,inplace=True) # drop last n rows
PARdata = PARdata.rename(columns={"0_0": "PAR_Wm2"})
PARdata = PARdata.reset_index()
PARdata = PARdata.rename(columns={"index": "text"})
PARdata['year'] = PARdata['text'].str[:4]
PARdata['month'] = PARdata['text'].str[5:7]
PARdata['day'] = PARdata['text'].str[8:10]
PARdata = PARdata.drop(['text'], axis=1)

# Combining PAR & TEMP
MeteoData = pd.DataFrame() 
MeteoData['Year_TEMP'] = TEMPdata['year']
MeteoData['Month_TEMP'] = TEMPdata['month']
MeteoData['Day_TEMP'] = TEMPdata['day']
MeteoData['Year_PAR'] = PARdata['year']
MeteoData['Month_PAR'] = PARdata['month']
MeteoData['Day_PAR'] = PARdata['day']
MeteoData['TEMP_K'] = TEMPdata['TEMP_K']
MeteoData['PAR_Wm2'] = PARdata['PAR_Wm2']

# Add the day of the year
MeteoData.insert(0, 'DOY', range(1, 1 + len(MeteoData)))

# Rescale between 1 and -1 for linear modelling purpose
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
temp = scaler.fit_transform(MeteoData[['DOY', 'TEMP_K', 'PAR_Wm2']])
dftemp = pd.DataFrame(temp, columns =['DOY','TEMP_K', 'PAR_Wm2'])
MeteoData[['DOY_rescaled','TEMP_K_rescaled', 'PAR_Wm2_rescaled']] = dftemp[['DOY','TEMP_K', 'PAR_Wm2']]
MeteoData

,DOY,Year_TEMP,Month_TEMP,Day_TEMP,Year_PAR,Month_PAR,Day_PAR,TEMP_K,PAR_Wm2,DOY_rescaled,TEMP_K_rescaled,PAR_Wm2_rescaled
0,1,2023,01,01,2023,01,01,280.681016,15.20566,-1.000000,0.070261,-0.996674
1,2,2023,01,02,2023,01,02,278.213197,14.774329,-0.994505,-0.082450,-0.998558
2,3,2023,01,03,2023,01,03,276.523374,49.361904,-0.989011,-0.187017,-0.847431
3,4,2023,01,04,2023,01,04,276.920893,185.98926,-0.983516,-0.162418,-0.250450
4,5,2023,01,05,2023,01,05,276.988134,185.25589,-0.978022,-0.158257,-0.253654
...,...,...,...,...,...,...,...,...,...,...,...,...
360,361,2023,12,27,2023,12,28,273.622099,16.583971,0.978022,-0.366550,-0.990651
361,362,2023,12,28,2023,12,29,274.698551,171.67815,0.983516,-0.299938,-0.312981
362,363,2023,12,29,2023,12,30,274.842937,151.94762,0.989011,-0.291004,-0.399192
363,364,2023,12,30,2023,12,31,273.848103,54.08896,0.994505,-0.352565,-0.826777


In [111]:
# Export to csv
MeteoData.to_csv('F:/Redaction/Bernadouze/Figures/MeteoData.csv',index=False)